In [2]:
import tunetrees.app.database
from tunetrees.app.database import SessionLocal, sqlalchemy_database_engine
from sqlalchemy.sql import text
from typing import Any, Dict, List
from sqlalchemy import Row, Table, func, select
from sqlalchemy.orm import Query, Session
from datetime import datetime, timedelta
from tabulate import tabulate
from sqlalchemy import Column, Float, ForeignKey, Index, Integer, Table, Text, text


from tunetrees.models.tunetrees import t_practice_list_joined, metadata

In [3]:
db = SessionLocal()

In [4]:
query: Query[Any] = db.query(t_practice_list_joined)
print(query)

SELECT practice_list_joined."ID" AS "practice_list_joined_ID", practice_list_joined."Title" AS "practice_list_joined_Title", practice_list_joined."Type" AS "practice_list_joined_Type", practice_list_joined."Structure" AS "practice_list_joined_Structure", practice_list_joined."Mode" AS "practice_list_joined_Mode", practice_list_joined."Incipit" AS "practice_list_joined_Incipit", practice_list_joined."Learned" AS "practice_list_joined_Learned", practice_list_joined."Practiced" AS "practice_list_joined_Practiced", practice_list_joined."Quality" AS "practice_list_joined_Quality", practice_list_joined."Easiness" AS "practice_list_joined_Easiness", practice_list_joined."Interval" AS "practice_list_joined_Interval", practice_list_joined."Repetitions" AS "practice_list_joined_Repetitions", practice_list_joined."ReviewDate" AS "practice_list_joined_ReviewDate", practice_list_joined."BackupPracticed" AS "practice_list_joined_BackupPracticed", practice_list_joined."NotePrivate" AS "practice_list_

In [5]:
order_by_query = query.order_by(
            func.DATE(t_practice_list_joined.columns.get("Practiced")).desc()
        )

In [6]:
def get_most_recent_review_date()->datetime:
    most_recent_schedualed  = query.order_by(
                func.DATE(t_practice_list_joined.columns.get("Practiced")).desc()
            ).limit(1)

    review_date_column_name = "ReviewDate"
    review_date = t_practice_list_joined.columns.get(review_date_column_name)
    assert review_date is not None
    review_date_column_index = list(t_practice_list_joined.columns.keys()).index(review_date_column_name)
    most_recent_review_date_str = most_recent_schedualed[0][review_date_column_index]
    most_recent_review_date = datetime.fromisoformat(most_recent_review_date_str)
    return most_recent_review_date
most_recent_review_date = get_most_recent_review_date()
print(most_recent_review_date)

2023-10-14 23:17:35


In [7]:
most_recent_review_date = get_most_recent_review_date()

scheduled_rows = (
    query.filter(
        func.DATE(t_practice_list_joined.columns.get("ReviewDate"))
        > (most_recent_review_date - timedelta(days=14))
    )
    .filter(
        func.DATE(t_practice_list_joined.columns.get("ReviewDate"))
        <= most_recent_review_date
    )
    .order_by(func.DATE(t_practice_list_joined.columns.get("ReviewDate")).desc())
    .offset(0)
    .limit(10)
    .all()
)

print(tabulate(scheduled_rows, headers=t_practice_list_joined.columns.keys()))

  ID  Title                   Type    Structure    Mode      Incipit                   Learned     Practiced              Quality    Easiness    Interval    Repetitions  ReviewDate           BackupPracticed      NotePrivate                                                                                                                          NotePublic                                                     Tags
----  ----------------------  ------  -----------  --------  ------------------------  ----------  -------------------  ---------  ----------  ----------  -------------  -------------------  -------------------  -----------------------------------------------------------------------------------------------------------------------------------  -------------------------------------------------------------  -----------
 808  Harvest Home            Hpipe   AABB         D Major   |D2FA D2FA|dfec dcBz|     2002-11-23  2023-09-27 23:17:35          5        1.7           17              